In [ ]:
## datetime 설치 필요, pip install datetime
## 자동화 tool인 selenium 라이브러리와 파일 설치 필요
## 구체적인 가이드는 구글에 '파이썬 셀레니움 설치' 검색 참조
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from datetime import datetime, date, timedelta
import pandas as pd
import re

In [ ]:
driver = webdriver.Chrome('/Users/koojaehong/Jupyter_notebook/chromedriver') # 창 띄우기

In [ ]:
driver.get('https://ceo.baemin.com/web/login') # 배달의 민족 사장님 로그인 페이지 이동

In [ ]:
## 동작 모음
def scroll_down():
    driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN) # 스크롤 내리기


## 데이터 추출 모음
# 리뷰 작성자 아이디 추출
def data_name(card_num):
    name = driver.find_element_by_xpath('//*[@id="review-page"]/div[4]/div/div['+str(card_num)+']/div[1]/div[1]/p').text
    
    return name
    
# 리뷰 별점 추출   
def data_star(card_num):
    star = driver.find_element_by_xpath('//*[@id="review-page"]/div[4]/div/div['+str(card_num)+']/div[1]/div[1]/div').text
    
    return star
                                        
# 리뷰 작성일 추출
def data_date(card_num):
    order_date = driver.find_element_by_xpath('//*[@id="review-page"]/div[4]/div/div['+str(card_num)+']/div[1]/div[1]/span').text
    
    if order_date == "오늘":
        order_date = str(date.today())
        
    elif order_date == "어제":
        order_date = str(date.today()-timedelta(days=1))
    
    elif order_date == "그제":
        order_date = str(date.today()-timedelta(days=2))
    
    elif order_date == "지난 주":
        order_date = str(date.today()-timedelta(weeks=1))
    
    elif order_date == "지난 달":
        order_date = str(date.today()-timedelta(weeks=4))
    
    elif order_date in ['2개월 전', '3개월 전', '4개월 전', '5개월 전', '6개월 전']:
        for month in range(2, 7):
            if order_date == str(month)+"개월 전":
                order_date = str(date.today()-timedelta(weeks=month*4))
            else:
                pass
    
    return order_date
                                        
# 리뷰 오른쪽 문단 추출
def right_paragraph(card_num):
    sentence= driver.find_element_by_xpath('//*[@id="review-page"]/div[4]/div/div['+str(card_num)+']/div[2]').text
    
    return sentence
    
# 리뷰 내용 추출
def review_paragraph(card_num):
    sentence = right_paragraph(card_num)
    
    try:
        sentence = re.sub('\n메뉴사진|메뉴사진', '', sentence)
        menu_location = sentence.find('주문메뉴')
        
        if menu_location==0:
            sentence = '해당내용 없음'
        else:
            sentence = sentence[:(menu_location-1)]
            sentence = re.sub('사장님에게만 보이는 리뷰입니다.|\n', '', sentence)
        
    except:
        pass
    
    return sentence

# 사장님 답변 제거                         
def delete_reply(card_num):
    sentence = right_paragraph(card_num)
    
    try:
        owner_location = sentence.rfind('\n사장님')
        
        if sentence.find('\n주문메뉴') < owner_location:
            sentence = sentence[:owner_location]
        else:
            pass
    
    except:
        pass
    
    return sentence
                                        
# 리뷰 주문 메뉴 추출                      
def menu_paragraph(card_num):
    try:
        sentence = delete_reply(card_num)
    except:
        pass
    
    menu_location = sentence.find('\n주문메뉴')
    sentence = sentence[(menu_location+5):]
    
    sentence = re.sub('\n등록된 지 30일이 지난 리뷰는 사장님 댓글을 등록할 수 없어요.|\n메뉴사진|\n주문메뉴|\n배달리뷰|\n좋아요|\n삭제|\n추천|\n사장님 댓글 등록하기', '', sentence)
    
    menu_list = sentence.split('\n')
    menu_list.remove('')
    
    menu_df = pd.DataFrame(data=menu_list, columns=['주문메뉴'])
    
    return menu_df

# 리뷰 모든 데이터 추출 함수
def card_df(card_num):
    df = pd.DataFrame(index=[0],columns=['작성자', '별점', '작성일', '작성내용'])
    df['작성자'] = data_name(card_num)
    df['별점'] = data_star(card_num)
    df['작성일'] = data_date(card_num)
    df['작성내용'] = review_paragraph(card_num)
    df = pd.concat([df, menu_paragraph(card_num)], axis=1)
    
    df['작성자'] = df['작성자'][0]
    df['별점'] = df['별점'][0]
    df['작성일'] = df['작성일'][0]
    df['작성내용'] = df['작성내용'][0]
    
    return df

In [ ]:
## 리뷰 추출 실행

review_df = pd.DataFrame() # 전체 리뷰 데이터를 담을 빈 데이터프레임 생성
for card_num in range(1,317): # card_num=리뷰 순서, 1부터 리뷰 개수+1만큼 입력합니다. ex)전체 리뷰 301개 -> 1, 302 입력
    if card_num % 9 == 0: # 9의 배수마다 스크롤 내리기
        scroll_down()
        scroll_down()
        scroll_down()
        scroll_down()
        scroll_down()
        scroll_down()
        scroll_down()
        time.sleep(0.5) # 0.5초 쉬어주기
    try:
        review_df = pd.concat([review_df, card_df(card_num)]) # 전체 리뷰 데이터프레임과 각 리뷰 데이터 합치기
    except:
        print(data_name(card_num)+"님의 주문메뉴는 없습니다") # 예외인 경우 : 주문메뉴가 없는 경우, 아이디+주문메뉴 없음 나타내기

In [ ]:
review_df.to_excel('배달의민족_리뷰테스트.xlsx') # 엑셀 형태로 추출
# 클렌징 : 주문메뉴에서 같이나온 텍스트는 제거
# 클렌징 : 작성일 기준을 날짜형태로 바꿀 것